In [2]:
# Importation des bibliothèques nécessaires
import os
import pickle
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Import données

In [3]:
def load_isot_dataset(fake_file, true_file):
    """Charge les fichiers de données et retourne un forFrame pour les données de fausses et vraies nouvelles."""
    fake_data = pd.read_csv(fake_file)
    true_data = pd.read_csv(true_file)
    
    fake_data['label'] = 0  # Label 0 pour les fausses nouvelles
    true_data['label'] = 1  # Label 1 pour les vraies nouvelles
    
    return pd.concat([fake_data, true_data], ignore_index=True)

# Exemple de chargement de données
train_isot = load_isot_dataset("data/Fake.csv", "data/True.csv")

# Diviser les données en caractéristiques (X) et labels (y)
X_isot = train_isot["text"]  # Le texte des articles
y_isot = train_isot["label"]  # Les labels (0 ou 1)

In [4]:
train_for = pd.read_csv("data/fake_or_real_news.csv")

# Remplacer "FAKE" par 0 et "REAL" par 1 dans la colonne "label"
train_for['label'] = train_for['label'].map({'FAKE': 0, 'REAL': 1})

# Diviser les données en caractéristiques (X) et labels (y)
X_for = train_for["text"]  # Le texte des articles
y_for = train_for["label"]  # Les labels (0 ou 1)

### Preprocess

#### BoW & TF-IDF

In [4]:
def preprocess_full(text):
    """Prétraitement complet pour les modèles Bag of Words et TF-IDF (avec lemmatisation)."""
    # Conversion en minuscules
    text = text.lower()
    
    # Suppression des URLs et handles Twitter
    text = re.sub(r"http\S+|www\S+|@\S+", "", text)
    
    # Suppression de la ponctuation et des espaces supplémentaires
    text = re.sub(r"[^\w\s]", "", text)
    
    # Suppression des espaces blancs supplémentaires
    text = re.sub(r"\s+", " ", text).strip()
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

In [5]:
def extract_bow_features(X_train, X_test):
    """Extrait les caractéristiques Bag of Words des jeux de données après prétraitement complet."""
    
    # Appliquer le prétraitement complet sur les textes
    X_train_cleaned = [preprocess_full(text) for text in X_train]
    X_test_cleaned = [preprocess_full(text) for text in X_test]
    
    # Initialisation du CountVectorizer
    vectorizer = CountVectorizer(stop_words='english', max_features=5000)
    
    # Transformation des données en matrices BoW
    X_train_bow = vectorizer.fit_transform(X_train_cleaned)
    X_test_bow = vectorizer.transform(X_test_cleaned)
    
    return X_train_bow, X_test_bow

In [6]:
def extract_tfidf_features(X_train, X_test):
    """Extrait les caractéristiques TF-IDF des jeux de données après prétraitement complet."""
    
    # Appliquer le prétraitement complet sur les textes
    X_train_cleaned = [preprocess_full(text) for text in X_train]
    X_test_cleaned = [preprocess_full(text) for text in X_test]
    
    # Initialisation du TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    
    # Transformation des données en matrices TF-IDF
    X_train_tfidf = vectorizer.fit_transform(X_train_cleaned)
    X_test_tfidf = vectorizer.transform(X_test_cleaned)
    
    return X_train_tfidf, X_test_tfidf

#### W2V & BERT & LC

In [38]:
def preprocess_light(text):
    """Prétraitement léger pour les modèles Word2Vec, BERT et Linguistic Cues (sans lemmatisation)."""
    # Conversion en minuscules
    text = text.lower()
    
    # Suppression des URLs et handles Twitter
    text = re.sub(r"http\S+|www\S+|@\S+", "", text)
    
    # Suppression des espaces blancs supplémentaires
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [27]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np

# Charger le modèle Word2Vec (évite de le télécharger à chaque exécution)
try:
    word2vec_model = KeyedVectors.load("word2vec-google-news-300.kv")  # Si déjà téléchargé
except FileNotFoundError:
    word2vec_model = api.load("word2vec-google-news-300")
    word2vec_model.save("word2vec-google-news-300.kv")  # Sauvegarde pour éviter le re-téléchargement

print(f"Word2Vec loaded! Vocab size: {len(word2vec_model.index_to_key)}")

Word2Vec loaded! Vocab size: 3000000


In [28]:
def extract_word2vec_features(X_train, X_test):
    """Extrait les caractéristiques Word2Vec des jeux de données après prétraitement léger."""
    
    def text_to_vec(text):
        """Convertit un texte en un vecteur moyen des embeddings de ses mots."""
        words = text.split()
        vecs = np.array([word2vec_model[word] for word in words if word in word2vec_model])
        return np.mean(vecs, axis=0) if len(vecs) > 0 else np.zeros(word2vec_model.vector_size)
    
    # Appliquer le prétraitement léger et vectoriser en une seule ligne
    X_train_cleaned = [preprocess_light(text) for text in X_train]
    X_test_cleaned = [preprocess_light(text) for text in X_test]
    
    X_train_word2vec = np.array([text_to_vec(text) for text in X_train_cleaned])
    X_test_word2vec = np.array([text_to_vec(text) for text in X_test_cleaned])
    
    return X_train_word2vec, X_test_word2vec

In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

model.eval()

In [ ]:
def extract_bert_features(X_train, X_test, batch_size=32):
    """Extrait les caractéristiques BERT en batch pour améliorer la vitesse."""
    
    def text_to_bert_vec(texts):
        """Convertir une liste de textes en vecteurs BERT (par batch)."""
        inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

    # Appliquer le prétraitement léger et transformer en batchs
    X_train_cleaned = [preprocess_light(text) for text in X_train]
    X_test_cleaned = [preprocess_light(text) for text in X_test]

    # Traiter en batchs pour accélérer
    X_train_bert = np.vstack([text_to_bert_vec(X_train_cleaned[i:i+batch_size]) for i in range(0, len(X_train_cleaned), batch_size)])
    X_test_bert = np.vstack([text_to_bert_vec(X_test_cleaned[i:i+batch_size]) for i in range(0, len(X_test_cleaned), batch_size)])
    
    return X_train_bert, X_test_bert

In [39]:
def extract_linguistic_cues(X_train, X_test):
    import re
    import numpy as np
    import pandas as pd
    import spacy

    # Charger le modèle spaCy anglais
    nlp = spacy.load("en_core_web_sm")

    def count_words(doc):
        return len([token for token in doc if token.is_alpha])

    def count_syllables(text):
        vowels = "aeiouyAEIOUY"
        return sum([sum(1 for char in word if char in vowels) for word in text.split()])

    def count_sentences(doc):
        return len(list(doc.sents))

    def count_long_words(doc, threshold=6):
        return sum(1 for token in doc if len(token.text) > threshold and token.is_alpha)

    def count_all_caps(doc):
        return sum(1 for token in doc if token.text.isupper())

    def count_unique_words(doc):
        return len(set(token.text.lower() for token in doc if token.is_alpha))

    def count_personal_pronouns(doc):
        pronouns = {'i', 'we', 'she', 'he', 'him', 'me', 'us'}
        return sum(1 for token in doc if token.text.lower() in pronouns)

    def count_articles(doc):
        articles = {'a', 'an', 'the'}
        return sum(1 for token in doc if token.text.lower() in articles)

    def count_pos_tags(doc, pos_tags):
        return sum(1 for token in doc if token.tag_ in pos_tags)

    def count_punctuation(text, p):
        return text.count(p)

    def compute_features(texts):
        features = []
        docs = list(nlp.pipe(texts))
        for doc, text in zip(docs, texts):
            wc = count_words(doc)
            sc = count_sentences(doc)
            data = {
                'word_count': wc,
                'syllables_count': count_syllables(text),
                'sentence_count': sc,
                'words_per_sentence': wc / max(sc, 1),
                'long_words_count': count_long_words(doc),
                'all_caps_count': count_all_caps(doc),
                'unique_words_count': count_unique_words(doc),
                'personal_pronouns%': count_personal_pronouns(doc) / max(wc, 1),
                'articles%': count_articles(doc) / max(wc, 1),
                'prepositions%': count_pos_tags(doc, {'IN'}) / max(wc, 1),
                'auxiliary_verbs%': count_pos_tags(doc, {'VB', 'VBP', 'VBG'}) / max(wc, 1),
                'common_adverbs%': count_pos_tags(doc, {'RB', 'RBR', 'RBS'}) / max(wc, 1),
                'conjunctions%': count_pos_tags(doc, {'CC'}) / max(wc, 1),
                'negations%': count_pos_tags(doc, {'RB'}) / max(wc, 1),
                'common_verbs%': count_pos_tags(doc, {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}) / max(wc, 1),
                'common_adjectives%': count_pos_tags(doc, {'JJ', 'JJR', 'JJS'}) / max(wc, 1),
                'punctuation_count': sum(count_punctuation(text, p) for p in ['.', ',', ':', ';', '!', '?', '-', '(', ')']),
                'full_stop_count': count_punctuation(text, '.'),
                'commas_count': count_punctuation(text, ','),
                'colons_count': count_punctuation(text, ':'),
                'semi_colons_count': count_punctuation(text, ';'),
                'question_marks_count': count_punctuation(text, '?'),
                'exclamation_marks_count': count_punctuation(text, '!'),
                'dashes_count': count_punctuation(text, '-'),
                'apostrophe_count': count_punctuation(text, "'"),
                'brackets_count': count_punctuation(text, '(') + count_punctuation(text, ')')
            }
            features.append(data)
        return pd.DataFrame(features)

    return compute_features(X_train), compute_features(X_test)

### Entrainement

In [40]:
# (X_train, X_test) = (X_isot, X_for)
# X_isot_BoW, _ = extract_bow_features(X_isot, X_for)
# X_isot_tfidf, _ = extract_tfidf_features(X_isot, X_for)

# X_for_BoW, _ = extract_bow_features(X_for, X_isot)
# X_for_tfidf, _ = extract_tfidf_features(X_for, X_isot)

# X_isot_W2V, _ = extract_word2vec_features(X_isot, X_for)
# X_for_W2V, _ = extract_word2vec_features(X_for, X_isot)

X_isot_LC, _ = extract_linguistic_cues(X_isot, X_for)
X_for_LC, _ = extract_linguistic_cues(X_for, X_isot)

In [ ]:
# import pickle

# # Sauvegarde des features dans des fichiers
# with open("X_isot_LC.pkl", "wb") as f:
#     pickle.dump(X_isot_LC, f)

# with open("X_for_LC.pkl", "wb") as f:
#     pickle.dump(X_for_LC, f)

In [5]:
def train_and_evaluate(X_train, y_train, X_test, y_test, model_type="svm"):
    """Entraîne un modèle sur 100% des données d'entraînement et évalue sur 100% des données de test."""
    # Initialisation du modèle en fonction du type
    if model_type == "svm":
        model = SVC(kernel="linear")
    elif model_type == "random_forest":
        model = RandomForestClassifier()
    elif model_type == "logistic_regression":
        model = LogisticRegression()
    elif model_type == "gradient_boosting":
        model = GradientBoostingClassifier()
    elif model_type == "adaboost":
        model = AdaBoostClassifier()
    elif model_type == "neural_network":
        model = MLPClassifier()

    # Entraîner le modèle sur toutes les données d'entraînement
    model.fit(X_train, y_train)

    # Faire des prédictions sur les données de test
    y_pred = model.predict(X_test)

    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy du modèle {model_type} : {accuracy}")

    # Création du dossier pour sauvegarder les modèles
    os.makedirs("cross_models_LC", exist_ok=True)

    # Sauvegarde du modèle
    model_filename = f"cross_models_LC/{model_type}_isot_LC.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(model, f)

    return model, accuracy

In [ ]:
# BOW

# Entraînement sur ISOT et évaluation sur FakeOrReal
# model_isot, accuracy_isot_to_for = train_and_evaluate(X_isot_BoW, y_isot, X_for_BoW, y_for, model_type="svm")
# Entraînement sur FakeOrReal et évaluation sur ISOT
# model_for, accuracy_for_to_isot = train_and_evaluate(X_for_BoW, y_for, X_isot_BoW, y_isot, model_type="svm")

Accuracy du modèle svm : 0.6553744042050871


In [ ]:
# TF-IDF

# Entraînement sur ISOT et évaluation sur FakeOrReal
# model_isot, accuracy_isot_to_for = train_and_evaluate(X_isot_tfidf, y_isot, X_for_tfidf, y_for, model_type="svm")
# Entraînement sur FakeOrReal et évaluation sur ISOT
# model_for, accuracy_for_to_isot = train_and_evaluate(X_for_tfidf, y_for, X_isot_tfidf, y_isot, model_type="svm")

Accuracy du modèle svm : 0.5327186066194485


In [ ]:
# W2V

# Entraînement sur ISOT et évaluation sur FakeOrReal
# model_isot, accuracy_isot_to_for = train_and_evaluate(X_isot_W2V, y_isot, X_for_W2V, y_for, model_type="svm")
# Entraînement sur FakeOrReal et évaluation sur ISOT
# model_for, accuracy_for_to_isot = train_and_evaluate(X_for_W2V, y_for, X_isot_W2V, y_isot, model_type="svm")

Accuracy du modèle svm : 0.7117911710989354


In [6]:
with open("X_isot_LC.pkl", "rb") as f:
    X_isot_LC = pickle.load(f)

with open("X_for_LC.pkl", "rb") as f:
    X_for_LC = pickle.load(f)

In [ ]:
# LC

# Entraînement sur ISOT et évaluation sur FakeOrReal
model_isot, accuracy_isot_to_for = train_and_evaluate(X_isot_LC, y_isot, X_for_LC, y_for, model_type="svm")
# Entraînement sur FakeOrReal et évaluation sur ISOT
# model_for, accuracy_for_to_isot = train_and_evaluate(X_for_LC, y_for, X_isot_LC, y_isot, model_type="svm")

Accuracy du modèle svm : 0.619403982360016


### Evaluation

In [11]:
# Charger et évaluer le modèle
def evaluate_model(X_test, y_test, model_path):
    """Évalue un modèle chargé depuis un fichier."""
    try:
        # Charger le modèle depuis le fichier pickle
        with open(model_path, "rb") as f:
            model = pickle.load(f)
    except FileNotFoundError:
        print(f"Erreur : fichier {model_path} introuvable.")
        return

    # Prédiction avec le modèle chargé
    y_pred = model.predict(X_test)

    # Afficher les résultats d'évaluation
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [12]:
# BoW

# Entraînement sur ISOT et évaluation sur FakeOrReal
evaluate_model(X_for_BoW, y_for, "cross_models_BoW/svm_isot_BoW.pkl")
# Entraînement sur FakeOrReal et évaluation sur ISOT
# evaluate_model(X_isot_BoW, y_isot, "cross_models_BoW/svm_for_BoW.pkl")

Accuracy: 0.5120757695343331
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.90      0.65      3164
           1       0.55      0.13      0.21      3171

    accuracy                           0.51      6335
   macro avg       0.53      0.51      0.43      6335
weighted avg       0.53      0.51      0.43      6335

Confusion Matrix:
 [[2839  325]
 [2766  405]]


In [ ]:
# TF-IDF

# Entraînement sur ISOT et évaluation sur FakeOrReal
evaluate_model(X_for_tfidf, y_for, "cross_models_tfidf/svm_isot_tfidf.pkl")
# Entraînement sur FakeOrReal et évaluation sur ISOT
evaluate_model(X_isot_BoW, y_isot, "cross_models_tfidf/svm_for_tfidf.pkl")